In [196]:
def convert_float_to_binary(float_value, precision):
    int_part_binary = bin(int(float_value))[2:]
    frac_part = float_value - int(float_value)
    frac_part_binary = ''
        
    while frac_part:
        frac_part *= 2
        if frac_part >= 1:
            frac_part_binary += '1'
            frac_part -= 1
        else:
            frac_part_binary += '0'

        if len(frac_part_binary) >= precision:
            break
            
    if frac_part_binary:
        return int_part_binary + '.' + frac_part_binary
    else:
        return int_part_binary

def normalize_binary(binary_str): 
    int_part, frac_part = binary_str.split('.')
    if int_part == '0':
        i = 0
        # Find the first '1' in the fractional part
        while i < len(frac_part) and frac_part[i] == '0':
            i += 1
        if i < len(frac_part):
            normalized_mantissa = '1.' + frac_part[i+1:]
            exponent = -(i+1)
        else:
            # Value is zero
            return '0.0', 0
        
    else:
        normalized_mantissa = '1.' + int_part[1:] + frac_part
        exponent = len(int_part) - 1
    
    return normalized_mantissa, exponent

def calculate_biased_exponent(exponent, bias):
    return exponent - bias

def round_mantissa(mantissa, precision):
    if mantissa[precision] == '0':
        return mantissa[:precision - 1]
    
    # str è immutabile in Python
    mantissa_list = list(mantissa)
    carry = True
    for i in reversed(range(precision)):
        if carry:
            if mantissa_list[i] == '0':
                mantissa_list[i] = '1'
                carry = False
            else:
                mantissa_list[i] = '0'
        else:
            break
            
    return ''.join(mantissa_list[:precision -1])

def convert_float_to_ieee754_representation(float_value, precision, lambda_val, omega_val, rounded = True):
    normalized_mantissa, exponent = normalize_binary(convert_float_to_binary(abs(float_value), precision))
    
    if not lambda_val <= exponent <= omega_val:
        raise ValueError(f"The exponent {exponent} does not fall between λ={lambda_val} and ω={omega_val}.")

    sign_bit = '0' if float_value >= 0 else '1'
    biased_exponent = calculate_biased_exponent(exponent, lambda_val)
    biased_exponent_bin = convert_float_to_binary(biased_exponent, precision).split('.')[0]  # prendiamo solo la parte intera
    
    if rounded:
        actual_mantissa = round_mantissa(normalized_mantissa.split('.')[1], precision)
    else:
        actual_mantissa = normalized_mantissa.split('.')[1][:precision - 1] # solo n-1 bit vengono effettivamente memorizzati in memoria
    
    return sign_bit + biased_exponent_bin + actual_mantissa

In [197]:
import unittest

class TestConvert(unittest.TestCase):
    def test_convert_integer_to_binary(self):
        self.assertEqual(convert_float_to_binary(13, 5), '1101')

    def test_convert_fraction_to_binary(self):
        self.assertEqual(convert_float_to_binary(0.9, 5), '0.11100')

    def test_convert_float_to_binary(self):
        self.assertEqual(convert_float_to_binary(13.9, 5), '1101.11100')

    def test_normalize_binary(self):
        self.assertEqual(normalize_binary('1101.11100'), ('1.10111100', 3))

    def test_calculate_biased_exponent(self):
        self.assertEqual(calculate_biased_exponent(3, -3), 6)
    
    def test_ieee754_truncated_representation(self):
        self.assertEqual(convert_float_to_ieee754_representation(-13.9, 5, -3, 4, False), '11101011')
        
    def test_ieee754_rounded_representation(self):
        self.assertEqual(convert_float_to_ieee754_representation(-13.9, 5, -3, 4), '11101100')
    
    def test_ieee754_out_of_bounds_exponent(self):
        with self.assertRaises(ValueError):
            convert_float_to_ieee754_representation(-13.9, 1, -1, 2, False)
        
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    unittest.TextTestRunner(verbosity=2).run(suite)
    
run_tests(TestConvert)

test_calculate_biased_exponent (__main__.TestConvert.test_calculate_biased_exponent) ... ok
test_convert_float_to_binary (__main__.TestConvert.test_convert_float_to_binary) ... ok
test_convert_fraction_to_binary (__main__.TestConvert.test_convert_fraction_to_binary) ... ok
test_convert_integer_to_binary (__main__.TestConvert.test_convert_integer_to_binary) ... ok
test_ieee754_out_of_bounds_exponent (__main__.TestConvert.test_ieee754_out_of_bounds_exponent) ... ok
test_ieee754_rounded_representation (__main__.TestConvert.test_ieee754_rounded_representation) ... ok
test_ieee754_truncated_representation (__main__.TestConvert.test_ieee754_truncated_representation) ... ok
test_normalize_binary (__main__.TestConvert.test_normalize_binary) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.012s

OK
